## Primero, leemos los datos y los mostramos

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import imageio
import numpy as np
from scipy import signal
from itertools import product
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm,PowerNorm

In [22]:
from importlib import reload

import isotropic_spectra
reload(isotropic_spectra)
from isotropic_spectra import wf_spectrum

import spectral_analysis
reload(spectral_analysis)
import spectral_analysis.tools
reload(spectral_analysis.tools)
from spectral_analysis.tools.spectral_analysis_tools import open_ds_kwe,calc_bm_igw_k

In [4]:
fname_var_fmt = "/home/antonio/Tesis/Datasets_compressed/{0}/{1}/{2}0_{3:05d}.npz"
fname_grid_fmt = "/home/antonio/Tesis/Datasets_compressed/{0}/hours/{1}.txt"
spectra_fn_fmt = "/home/antonio/Tesis/spectra/{0}/{1}_{2}_{3}.npz"

In [5]:
idx_t_JFM_days = [i for i in range(377) if i>=110 and i<201]
idx_t_JAS_days = [i for i in range(377) if i>=292]
idx_t_JFM_hours = [(24*i)+hr for i in range(377) if i>=110 and i<201 for hr in range(24)]
idx_t_JAS_hours = [(24*i)+hr for i in range(377) if i>=292  for hr in range(24)][:-18]
idx_t = {
	"days": {
		"JFM": idx_t_JFM_days,
		"JAS": idx_t_JAS_days,
		"JFMJAS": idx_t_JFM_days+idx_t_JAS_days
	},
	"hours": {
		"JFM": idx_t_JFM_hours,
		"JAS": idx_t_JAS_hours,
		"JFMJAS": idx_t_JFM_hours+idx_t_JAS_hours
	}
}
all_t_res = list(idx_t.keys())

In [6]:
ids_Cal = [762, 787, 809, 831, 852, 868] # California -- 23 to 51 N
ids_Can = [709, 730, 750, 771] # Canarias -- 16 to 36 N
ids_Peru = [450, 572, 596, 616, 636] # Peru Chile -- 5 to 45 S
ids_Ben = [459, 533, 556, 578, 602] # Benguela -- 15 to 37 S ** Quitamos la **459** y ponemos la 602
ids_Kuro = [733, 751, 796]
ids_regions = {
	"California": ids_Cal,
	"Canarias": ids_Can,
	"Peru": ids_Peru[:-1],
	"Benguela": ids_Ben,
	"Kuroshio": ids_Kuro
}

In [7]:
fname_var_fmt2 = "/home/antonio/Tesis/Datasets/{0}/{1}/{2}0_{3:05d}.txt"
#fname_grid_fmt = "/home/antonio/Tesis/Datasets/{0}/{1}/{2}.txt"
def UV4id(id,season,t_res="hours",t_firstaxis=False):
	time = idx_t[t_res][season]
	
	for idx,t in enumerate(time):
		if t_res=="hours":
			U_ = np.load(fname_var_fmt.format(id,t_res,"U",t))["uv"]
			V_ = np.load(fname_var_fmt.format(id,t_res,"V",t))["uv"]
		else:
			U_ = np.loadtxt(fname_var_fmt2.format(id,t_res,"U",t))
			V_ = np.loadtxt(fname_var_fmt2.format(id,t_res,"V",t))
		if idx==0:
			print(id,t)
			#print(U_.shape)
			shape_uv = U_.shape
			shape = (shape_uv[0],shape_uv[1],len(time))
			print("UV shape",shape)
			U = np.zeros(shape)
			V = np.zeros(shape)
		U[:,:,idx] = U_
		V[:,:,idx] = V_

	if t_firstaxis:
		U = np.moveaxis(U,-1,0)
		V = np.moveaxis(V,-1,0)

	return U,V

In [8]:
def dxdyf4id(id,t_res="hours"):
	dx_ = np.loadtxt(fname_grid_fmt.format(id,"dx"))
	dy_ = np.loadtxt(fname_grid_fmt.format(id,"dy"))
	f_ = np.loadtxt(fname_grid_fmt.format(id,"f"))

	return dx_,dy_,f_

In [9]:
def dxdy_avg(dx,dy):
	dx_avg = np.mean(dx.flatten())
	dy_avg = np.mean(dy.flatten())
	return dx_avg,dy_avg

In [10]:
def d_dx(f,dx):
	return (np.gradient(f,axis=1,edge_order=2)/dx)

def d_dy(f,dy):
	return (np.gradient(f,axis=0,edge_order=2)/dy)

In [11]:
def div_(u,v,dx,dy):
	return d_dx(u,dx) + d_dy(v,dy)

In [12]:
def rv_(u,v,dx,dy):
	return d_dx(v,dx) - d_dy(u,dy)

In [13]:
def st_(u,v,dx,dy):
	sn = d_dx(u,dx) - d_dy(v,dy)
	ss = d_dx(v,dx) + d_dy(u,dy)
	return np.sqrt( np.square(sn) + np.square(ss) )

In [14]:
def ow_(u,v,dx,dy):
	return np.square(st_(u,v,dx,dy)) - np.square(rv_(u,v,dx,dy))

In [15]:
def detrend(S):
	S = signal.detrend(S,axis=0,type='linear')
	S = signal.detrend(S,axis=1,type='linear')
	S = signal.detrend(S,axis=2,type='linear')
	return S

In [16]:
def centerIdx(length):
	if length%2==1:
		return int((length-1)/2+1)
	else:
		return int(length/2)

In [17]:
def fig2img(fig):
	fig.canvas.draw()
	image = np.frombuffer(fig.canvas.tostring_rgb(),dtype='uint8')
	image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
	return image

In [18]:
def calc_ispec(E,k,l,om):
	""" calculates isotropic spectrum from 3D spectrum """

	print("Input shapes: E,k,l,f",E.shape,k.shape,l.shape,om.shape)

	dk,dl = k[1]-k[0],l[1]-l[0]
	l,k = np.meshgrid(l,k)
	wv = np.sqrt(k**2 + l**2)
	print("Grid shapes: k,l,K",k.shape,l.shape,wv.shape)

	if k.max()>l.max():
		kmax = l.max()
		dkr = dl
	else:
		kmax = k.max()
		dkr = dk

	y,x = np.indices(E.shape[:-1])
	print("Indices shape: x,y",x.shape,y.shape)
	cx = centerIdx(np.max(x)+1)
	cy = centerIdx(np.max(y)+1)
	kr_bins_flat = (np.sqrt((x-cx)**2 + (y-cy)**2)).astype(np.int).ravel() # Medimos la distancia en múltiplos de dkr
	nr = np.bincount(kr_bins_flat)
	nbins = int(np.round(kmax/dkr))
	print("Nbins",nbins)

	# create radial wavenumber
	#dkr = np.sqrt(dk**2 + dl**2)
	#kr =  np.arange(nbins)*dkr # Bin edges
	kr =  np.arange(nbins)*dkr+dkr/2 # Bin centers

	for iw in range(om.size):
		E_ = E[:,:,iw]


		# Isotropization w/ radial profile (See https://stackoverflow.com/questions/21242011/most-efficient-way-to-calculate-radial-profile)
		Ei = (np.bincount(kr_bins_flat,E_.ravel())/nr)[:nbins]

		if True:
			if (iw-1)%5==0:
				plt.clf()
				fig, ax = plt.subplots(1,2,figsize=(15,6))
				ax = ax.flat[:]
				plt.sca(ax[0])
				plt.pcolormesh(k,l,E_,cmap='rainbow',norm = LogNorm())
				plt.colorbar()
				plt.sca(ax[1])
				plt.semilogy(kr,Ei)
				plt.suptitle("Freq = {0:0.4f} cph, T = {1:0.1f} hours".format(om[iw],1/om[iw]),size="xx-large")
				plt.show()

		# Isotropization - Original way
		#Ei = np.zeros(kr.size)
		#for ik in range(kr.size):
		#    Kr_min = kr[ik]-dkr/2
		#    Kr_max = kr[ik]+dkr/2
		#    fkr =  (wv>=Kr_min) & (wv<Kr_max)
		#    dth = np.pi / (fkr.sum()-1)
		#    #Area_ik = np.pi*(Kr_max**2 - Kr_min**2)
		#    Ei[ik] = E_[fkr].sum() * kr[ik] * dth #np.mean(E_[fkr])

		# Storage
		if iw == 0:
			Eiso = np.zeros((nbins,om.size))
		Eiso[:,iw] = Ei

	return kr[:nbins], Eiso[:nbins,:]

In [19]:
def calculate_save_spectrum(region_id,season,t_res="hours"):
	assert t_res in all_t_res,"t_res not in [hours,days]"
	dt_hr = 1 if t_res=="hours" else 24

	U,V = UV4id(region_id,season,t_res)
	N = U.size
	print("U,V shapes",U.shape,V.shape," -- total elements:",N)
	U,V = detrend(U),detrend(V)
	print("U,V size in MB",U.nbytes/(1024**2),V.nbytes/(1024**2))

	DX,DY,F = dxdyf4id(region_id)
	dx,dy = dxdy_avg(DX,DY) # En metros
	dx,dy,dt = dx/1000,dy/1000,dt_hr # km,km,hr
	print("dx,dy,dt",dx,dy,dt)
	e_U = np.sum(U**2)#*dx*dy*dt
	e_V = np.sum(V**2)#*dx*dy*dt
	print("U,V energy (x,y,t)",e_U,e_V)

	beta = 1.33/2
	EU,k,l,om = wf_spectrum.spec_est3(U,dx,dy,dt,beta=beta)
	EV,_,_,_ = wf_spectrum.spec_est3(V,dx,dy,dt,beta=beta)
	del U
	del V

	dk,dl,dom = k[1]-k[0],l[1]-l[0],om[1]-om[0]
	se_U = np.sum(EU)/N #*dk*dl*dom
	se_V = np.sum(EV)/N #*dk*dl*dom
	print("U,V energy (k,l,f)",se_U,se_V)
	eseU_r = e_U/se_U
	eseV_r = e_V/se_V
	print("Ratios E(x,y,t)/E(k,l,f) for U,V",eseU_r,eseV_r)
	#EU = EU*(eseU_r)
	#EV = EV*(eseV_r)
	#se_U = np.sum(EU)/N #*dk*dl*dom
	#se_V = np.sum(EV)/N #*dk*dl*dom
	#print("Corrected U,V energy (k,l,f)",se_U,se_V)
	print("Dividing by N (total # of elems) to get (approximate) PSD")
	EU = EU/N
	EV = EV/N

	E = 0.5*(EU+EV)
	del EU
	del EV

	print("k,l,om shapes",k.shape,l.shape,om.shape)
	print
	print("sU,sV shapes",E.shape,E.shape)
	print("min,max sE",np.min(E),np.max(E))
	#print("FFT x scales (km)",["{:.2f}".format(1/k_) for k_ in k])
	#print("FFT y scales (km)",["{:.2f}".format(1/l_) for l_ in l])
	#print("FFT t scales (h)",["{:.2f}".format(1/om_) for om_ in om])
	#ku_iso, EU_iso = calc_ispec(EU,ku,lu,omu)
	#kv_iso, EV_iso = calc_ispec(EV,kv,lv,omv)
	#print("k,omega,sU_iso shapes",ku_iso.shape,omu.shape,EU_iso.shape)
	#print("k,omega,sv_iso shapes",kv_iso.shape,omv.shape,EV_iso.shape)
	print("Spectrum isotropization")
	k_iso, E_iso = calc_ispec(E,k,l,om)
	#print("Horizontal scales (km)",["{:.2f}".format(1/ki_) for ki_ in k_iso])
	del E

	fname = spectra_fn_fmt.format(region_id,"KE",season,t_res)
	os.makedirs(os.path.dirname(fname), exist_ok=True)
	np.savez_compressed(fname,k=k_iso,om=om,S=E_iso)
	print("Saved",fname)

In [20]:
def plot_spectra(region_id,seasons,t_res="hours"):
	assert t_res in all_t_res,"t_res not in [hours,days]"
	
	fig, ax = plt.subplots(1,2,figsize=(15,6))
	ax = ax.flat[:]

	for j,season in enumerate(seasons):
		plt.sca(ax[j])

		fname = spectra_fn_fmt.format(region_id,"KE",season)
		print("Loading",fname)
		s_loaded = np.load(fname)
		k,om,S = s_loaded["k"],s_loaded["om"][:-150],s_loaded["S"][:,:-150]
		#print("Horizontal scales (km)",["{:.2f}".format(1/k_) for k_ in k])
		#print("Time scales (h)",["{:.2f}".format(1/om_) for om_ in om])
		print("Spectra shape",S.shape)

		# Plot
		plt.pcolormesh(k,om,(om*S).T*k,cmap='rainbow',norm = LogNorm())

		plt.clim([1e-6,1e-2])
		plt.xscale('log')
		plt.xlim([k[1],k[-1]/2.5])
		plt.xticks([1/10,1/50,1/100,1/200],['10','50','100','200'])
		plt.yscale('log')
		plt.ylim([om[1],om[-100]])
		plt.yticks([1/3,1/6,1/12,1/24,1/(24*7),1/(24*30)],['3 h','6 h','12 h','1 d','1 w','1 mo'])
		plt.colorbar()
		plt.title("{}/{}".format(region_id,season))

	plt.show()

In [ ]:
#plt.figure()
calculate_save_spectrum(762,"JAS")
#plt.figure()
#calculate_save_spectrum(762,"JFM")

762 7008
UV shape (288, 289, 2022)


In [ ]:
plot_spectra(762,["JFM","JAS"])

In [ ]:
for current,r_ids in ids_regions.items():
    if current=="Benguela" or current=="Peru":
        for season in idx_t.keys():
            print(current,season)
            for r_id in r_ids:
                calculate_save_spectrum(r_id,season)

In [ ]:
for r_id in ids_regions["California"]:
    plot_spectra(r_id,["JAS","JFM"])

In [ ]:
for r_id in ids_regions["Canarias"]:
    plot_spectra(r_id,["JAS","JFM"])

In [ ]:
for r_id in ids_regions["Peru"]:
    plot_spectra(r_id,["JAS","JFM"])

In [ ]:
for r_id in ids_regions["Benguela"]:
    plot_spectra(r_id,["JAS","JFM"])

In [ ]:
for r_id in ids_regions["Kuroshio"]:
    plot_spectra(r_id,["JAS","JFM"])

In [ ]:
%%script false
%%javascript
var cell = Jupyter.notebook.get_selected_cell();
var config = cell.config;
var patch = {
      CodeCell:{
        cm_config:{indentUnit: 4} // only change here.
      }
    }
config.update(patch)

// IPython.notebook.save_notebook()

## Histogramas 2D de vorticidad-estiramiento

In [ ]:
zrange=(-3.5,3.5)
srange=(0,7)
drange=(-3.5,3.5)
zbins=np.linspace(*zrange)
sbins=np.linspace(*srange)
dbins=np.linspace(*drange)

def zeta_sigma_hist(r_id,season,t_res="days"):
	U,V = UV4id(r_id,season,t_res,t_firstaxis=True)
	DX,DY,F = dxdyf4id(r_id,t_res) # De hecho t_res no importa, porque es un valor fijo
	
	zeta = rv_(U,V,DX,DY)/F
	#delta = div_(U,V,DX,DY)/F
	sigma = np.abs(st_(U,V,DX,DY)/F)
	ow = ow_(U,V,DX,DY)/(F**2)
	Q0 = np.std(ow)
	sigma_elip = np.sqrt(np.square(zbins)-Q0)
	sigma_hyp = np.sqrt(np.square(zbins)+Q0)
	
	
	#plt.figure(figsize=(15,6))
	#plt.subplot(1,2,1)
	plt.hist2d(zeta.flat[:],sigma.flat[:],bins=[zbins,sbins],density=True,norm=LogNorm(vmin=1e-5),cmap=plt.cm.cubehelix_r)
	plt.colorbar()
	plt.plot(zbins,sigma_elip,color='darkturquoise',linewidth=2)
	plt.plot(zbins,sigma_hyp,color='lightcoral',linewidth=2)
	xlims = list(plt.xlim(zrange))
	ylims = list(plt.ylim(srange))
	yplus = xlims
	yminus = [-1*y for y in yplus]
	#plt.plot([0,0],ylims,linestyle=":",color='k',scalex=False,scaley=False) # Vertical line
	#plt.plot(xlims,yplus,linestyle='--',color='w', scalex=False, scaley=False)
	#plt.plot(xlims,yminus,linestyle='--',color='w', scalex=False, scaley=False)
	plt.xlabel("$\zeta/f$",size="xx-large")
	plt.ylabel("$\sigma/f$",size="xx-large")
	#plt.subplot(1,2,2)
	#plt.hist2d(zeta.flat[:],delta.flat[:],bins=100,density=True,norm=LogNorm(vmin=1e-4),cmap=plt.cm.gnuplot2_r)
	#plt.colorbar()
	#xlims = list(plt.xlim(zrange))
	#ylims = list(plt.ylim(drange))
	#plt.plot([0,0],ylims,linestyle=":",color='k',scalex=False,scaley=False) # Vertical line
	#plt.plot(xlims,[0,0],linestyle=":",color='k',scalex=False,scaley=False) # Horizontal line
	#plt.xlabel("$\zeta/f$",size="xx-large")
	#plt.ylabel("$\delta/f$",size="xx-large")
	plt.title("{} / {}".format(r_id,season),size="xx-large")
	#plt.show()

In [ ]:
def plot_2dhists(current,t_res="days"):
	for r_id in ids_regions[current]:
		fig, ax = plt.subplots(1,2,figsize=(12,5))
		ax = ax.flat[:]
		for i,season in enumerate(["JFM","JAS"]):
			plt.sca(ax[i])
			zeta_sigma_hist(r_id,season,t_res)
		plt.show()

In [ ]:
plot_2dhists("California")

In [ ]:
plot_2dhists("Canarias")

In [ ]:
plot_2dhists("Peru")

In [ ]:
plot_2dhists("Benguela")

In [ ]:
plot_2dhists("Kuroshio")